In [36]:
"""
01_build_foundations.ipynb
───────────────────────────
Objetivo:
Construir los cuatro DataFrames globales del proyecto PIVI a partir de:
- all_results (diccionario por industria con métricas y rutas)
- dependencias{año}.csv.gz (consolidado 9.3M filas)

Salidas (en /data/processed/dependencias_consolidadas/):
1) intermediarios_globales.parquet
2) country_profiles.parquet
3) relaciones_criticas.parquet
4) caminos_significativos.parquet
"""

from pathlib import Path
import pandas as pd
import numpy as np
import pickle
import gzip
from collections import defaultdict, Counter

# Opcional: asegurar que librerías numéricas no saturan hilos
import os
os.environ.setdefault("OPENBLAS_NUM_THREADS", "1")
os.environ.setdefault("MKL_NUM_THREADS", "1")
os.environ.setdefault("OMP_NUM_THREADS", "1")
os.environ.setdefault("NUMEXPR_NUM_THREADS", "1")

pd.set_option("display.max_columns", 120)
pd.set_option("display.width", 160)



In [37]:
# Ajusta esta base si tu notebook está en otra carpeta
base_path = Path.cwd().parent.parent
processed_path = base_path / "data" / "processed" / "dependencias_consolidadas"

for p in [processed_path]:
    p.mkdir(parents=True, exist_ok=True)
    print("📁 Ruta:", p, "→ existe:", p.exists())



📁 Ruta: c:\Users\Usuario\Documents\Github\Seguridad Economica\data\processed\dependencias_consolidadas → existe: True


In [38]:
# Intento 1: cargar dependencias{año}.csv.gz más reciente
candidates = sorted(processed_path.glob("dependencias*.csv.gz"))
if not candidates:
    raise FileNotFoundError("No encuentro dependencias*.csv.gz en dependencias_consolidadas")
csv_path = candidates[-1]  # el más reciente por orden alfabético
print("🗂️ Consolidado detectado:", csv_path.name)

# Carga del consolidado
df = pd.read_csv(csv_path, sep=";")
expected_cols = {"industry","dependent_country","supplier_country","dependency_value","direct_dependency","indirect_dependency","trade_value","longitud_optima"}
missing = expected_cols - set(df.columns)
if missing:
    raise ValueError(f"Faltan columnas en el consolidado: {missing}")
print("✅ dependencias cargado:", df.shape)

# Intento 2: localizar all_results (pkl o parquet)
pkl_path = processed_path / "all_results.pkl"
pq_path  = processed_path / "all_results.parquet"

if pkl_path.exists():
    with open(pkl_path, "rb") as f:
        all_results = pickle.load(f)
    print("✅ all_results.pkl cargado con", len(all_results), "industrias")
elif pq_path.exists():
    # Si guardaste como parquet por industria, ajusta a tu formato; aquí asumimos .pkl
    raise FileNotFoundError("Tienes all_results.parquet pero este notebook espera .pkl (ajusta la carga si procede).")
else:
    raise FileNotFoundError("No encuentro all_results.pkl ni all_results.parquet en dependencias_consolidadas")


🗂️ Consolidado detectado: dependencias2022_borrar.csv.gz
✅ dependencias cargado: (7198964, 8)
✅ all_results.pkl cargado con 170 industrias


In [39]:
"""
Objetivo:
Construir el DataFrame intermediarios_globales.parquet a partir de all_results.

Define, para cada país:
- frequency_total  → cuántas veces aparece como intermediario
- strength_total   → suma de fuerzas ponderadas de los caminos donde es intermediario
- global_score     → score combinado (0.4 frecuencia normalizada + 0.6 fuerza normalizada)
- global_rank      → ranking global por score
"""

from collections import defaultdict

freq_counter = defaultdict(int)
strength_counter = defaultdict(float)

# ==========================
#   AGREGAR TODAS LAS INDUSTRIAS
# ==========================

for industry, data in all_results.items():

    # Validar estructura esperada
    if "intermediary_frequency" not in data["results"]:
        print(f"⚠️ 'intermediary_frequency' no encontrado en {industry}")
        continue
    if "intermediary_strength" not in data["results"]:
        print(f"⚠️ 'intermediary_strength' no encontrado en {industry}")
        continue

    freq_dict = data["results"]["intermediary_frequency"]
    strength_dict = data["results"]["intermediary_strength"]

    # Acumular frecuencia y fuerza por país
    for country, f in freq_dict.items():
        freq_counter[country] += f

    for country, s in strength_dict.items():
        strength_counter[country] += s


# ==========================
#   CREAR DATAFRAME
# ==========================

intermediarios_globales = pd.DataFrame({
    "country": list(freq_counter.keys()),
    "frequency_total": [freq_counter[c] for c in freq_counter.keys()],
    "strength_total": [strength_counter[c] for c in freq_counter.keys()],
})

# ==========================
#   NORMALIZAR Y CALCULAR SCORE
# ==========================

if intermediarios_globales["frequency_total"].max() > 0:
    intermediarios_globales["freq_norm"] = (
        intermediarios_globales["frequency_total"] /
        intermediarios_globales["frequency_total"].max()
    )
else:
    intermediarios_globales["freq_norm"] = 0

if intermediarios_globales["strength_total"].max() > 0:
    intermediarios_globales["strength_norm"] = (
        intermediarios_globales["strength_total"] /
        intermediarios_globales["strength_total"].max()
    )
else:
    intermediarios_globales["strength_norm"] = 0

intermediarios_globales["global_score"] = (
    0.4 * intermediarios_globales["freq_norm"] +
    0.6 * intermediarios_globales["strength_norm"]
)

intermediarios_globales = intermediarios_globales.sort_values(
    "global_score", ascending=False
).reset_index(drop=True)

intermediarios_globales["global_rank"] = intermediarios_globales.index + 1

# Eliminar las columnas auxiliares de normalización si no las quieres
intermediarios_globales = intermediarios_globales[[
    "country", "frequency_total", "strength_total", "global_score", "global_rank"
]]

# ==========================
#   EXPORTAR
# ==========================

parquet_path = processed_path / "intermediarios_globales.parquet"
intermediarios_globales.to_parquet(parquet_path, index=False)

print("✅ intermediarios_globales PARQUET generado correctamente")
print("   →", parquet_path)
print("   Total países:", len(intermediarios_globales))



✅ intermediarios_globales PARQUET generado correctamente
   → c:\Users\Usuario\Documents\Github\Seguridad Economica\data\processed\dependencias_consolidadas\intermediarios_globales.parquet
   Total países: 236


In [40]:
"""
Construye un perfil por país:
- vulnerability: media ponderada (por trade_value) de dependency_value cuando el país es importador
- importance:   media ponderada (por trade_value) de dependency_value cuando el país es exportador
- indirect_share: media ponderada de (indirect_dependency / dependency_value) cuando dependency_value>0
- num_suppliers_effective: número efectivo de proveedores (1 / sum p_i^2), con p_i = peso del proveedor en la dependencia
- intermediary_rank: rank global desde intermediarios_globales
"""

df_imp = df.copy()

# Vulnerabilidad (como importador)
def weighted_mean(x, w):
    s = w.sum()
    return (x*w).sum()/s if s > 0 else np.nan

vul = (
    df_imp
    .groupby("dependent_country")
    .apply(lambda g: weighted_mean(g["dependency_value"], g["trade_value"]))
    .rename("vulnerability")
    .reset_index()
)

# Importancia (como exportador)
imp = (
    df_imp
    .groupby("supplier_country")
    .apply(lambda g: weighted_mean(g["dependency_value"], g["trade_value"]))
    .rename("importance")
    .reset_index()
    .rename(columns={"supplier_country":"country"})
)

# Share indirecto (ponderado)
df_pos = df_imp[df_imp["dependency_value"] > 0].copy()
df_pos["indirect_share_aux"] = df_pos["indirect_dependency"] / df_pos["dependency_value"]
ind_share = (
    df_pos
    .groupby("dependent_country")
    .apply(lambda g: weighted_mean(g["indirect_share_aux"], g["trade_value"]))
    .rename("indirect_share")
    .reset_index()
)

# Número efectivo de proveedores (ENP/HHI inverso) por país importador
def effective_suppliers(group: pd.DataFrame) -> float:
    # p_i = peso del proveedor en la dependencia total del importador (ponderado por valor de comercio)
    g = group.copy()
    # peso por fila = trade_value * dependency_value (aprox. contribución dependiente ponderada)
    g["w"] = g["trade_value"] * g["dependency_value"]
    s = g["w"].sum()
    if s <= 0:
        return np.nan
    shares = g.groupby("supplier_country")["w"].sum() / s
    return 1.0 / (shares.pow(2).sum())  # ENP

enp = (
    df_imp
    .groupby("dependent_country")
    .apply(effective_suppliers)
    .rename("num_suppliers_effective")
    .reset_index()
)

# Intermediary rank
inter_rank = intermediarios_globales[["country","global_rank"]]

# Merge todo a un único perfil por país
country_profiles = (
    vul
    .rename(columns={"dependent_country":"country"})
    .merge(imp, on="country", how="outer")
    .merge(ind_share.rename(columns={"dependent_country":"country"}), on="country", how="left")
    .merge(enp.rename(columns={"dependent_country":"country"}), on="country", how="left")
    .merge(inter_rank, on="country", how="left")
    .sort_values("country")
    .reset_index(drop=True)
)

out_path = processed_path / "country_profiles.parquet"
country_profiles.to_parquet(out_path, index=False)
print("✅ country_profiles:", country_profiles.shape, "→", out_path)
country_profiles.head(10)


C:\Users\Usuario\AppData\Local\Temp\ipykernel_23692\1061405592.py:20: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  .apply(lambda g: weighted_mean(g["dependency_value"], g["trade_value"]))
C:\Users\Usuario\AppData\Local\Temp\ipykernel_23692\1061405592.py:29: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  .apply(lambda g: weighted_mean(g["dependency_value"], g["trade_value"]))
C:\Users\Usuario\AppData\Local\T

✅ country_profiles: (236, 6) → c:\Users\Usuario\Documents\Github\Seguridad Economica\data\processed\dependencias_consolidadas\country_profiles.parquet


C:\Users\Usuario\AppData\Local\Temp\ipykernel_23692\1061405592.py:61: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  .apply(effective_suppliers)


,country,vulnerability,importance,indirect_share,num_suppliers_effective,global_rank
0,ABW,0.516792,0.167510,0.132497,2.813917,187
1,AFG,0.577308,0.160276,0.130087,6.206743,135
2,AGO,0.315654,0.060304,0.246660,8.482180,112
3,AIA,0.797790,0.005993,0.080403,2.834298,210
4,ALB,0.331551,0.080671,0.264143,6.694662,129
5,AND,0.626569,0.014901,0.183066,1.518560,200
6,ARE,0.314438,0.289046,0.342378,3.957941,14
7,ARG,0.386931,0.293693,0.306286,4.504782,39
8,ARM,0.423159,0.149217,0.259367,3.577078,127
9,ASM,0.649449,0.005569,0.187900,4.523798,215


In [41]:
"""
Identifica relaciones críticas por industria:
- dependencia_total >= threshold (por defecto 0.7)
- menos de min_paths caminos alternativos (por defecto 3)
Devuelve un parquet con (exportador, importador, dependencia_total, caminos_alternativos, criticidad, industry)
"""

threshold = 0.7
min_paths = 3

rows = []
for industry, payload in all_results.items():
    res = payload["results"]
    deps = res.get("dependencies", [])
    crit_inter = res.get("critical_intermediaries", {})  # dict: "EXP->IMP" -> list(paths)
    for d in deps:
        exp = d["exportador"]
        imp = d["importador"]
        dt  = d["dependencia_total"]
        if dt >= threshold:
            key = f"{exp}->{imp}"
            alt = len(crit_inter.get(key, []))
            if alt < min_paths:
                criticidad = 1.0 - (alt / min_paths)
                rows.append({
                    "exportador": exp,
                    "importador": imp,
                    "dependencia_total": float(dt),
                    "caminos_alternativos": int(alt),
                    "criticidad": float(criticidad),
                    "industry": industry
                })

relaciones_criticas = pd.DataFrame(rows).sort_values(["criticidad","dependencia_total"], ascending=[False,False]).reset_index(drop=True)

out_path = processed_path / "relaciones_criticas.parquet"
relaciones_criticas.to_parquet(out_path, index=False)
print("✅ relaciones_criticas:", relaciones_criticas.shape, "→", out_path)
relaciones_criticas.head(10)


✅ relaciones_criticas: (7554, 6) → c:\Users\Usuario\Documents\Github\Seguridad Economica\data\processed\dependencias_consolidadas\relaciones_criticas.parquet


,exportador,importador,dependencia_total,caminos_alternativos,criticidad,industry
0,JPN,ARE,1.0,0,1.0,154 Manufacturing services on physical inputs
1,ITA,PHL,1.0,0,1.0,154 Manufacturing services on physical inputs
2,GBR,SAU,1.0,0,1.0,154 Manufacturing services on physical inputs
3,GBR,UKR,1.0,0,1.0,154 Manufacturing services on physical inputs
4,JPN,VNM,1.0,0,1.0,154 Manufacturing services on physical inputs
5,JPN,ARE,1.0,0,1.0,155 Maintenance and repair services n.i.e.
6,USA,BRN,1.0,0,1.0,155 Maintenance and repair services n.i.e.
7,USA,COL,1.0,0,1.0,155 Maintenance and repair services n.i.e.
8,USA,CRI,1.0,0,1.0,155 Maintenance and repair services n.i.e.
9,JPN,CYM,1.0,0,1.0,155 Maintenance and repair services n.i.e.


In [42]:
"""
Agrega rutas críticas (critical_paths) que aparecen en múltiples industrias.
Agrupa por (exportador, importador, intermediarios) y suma la fuerza; además cuenta en cuántas industrias aparece.
"""

# Normalizar representación de intermediarios como tupla ordenada (para agrupar idénticos)
def normalize_interms(lst):
    # mantenemos el orden del camino tal y como viene (impacta interpretación); si quieres set ordenado, usa tuple(sorted(lst))
    return tuple(lst)

agg = {}

for industry, payload in all_results.items():
    res = payload["results"]
    for path in res.get("critical_paths", []):
        exp = path["exportador"]
        imp = path["importador"]
        inters = normalize_interms(path["intermediarios"])
        fuerza = float(path["fuerza"])
        key = (exp, imp, inters)
        if key not in agg:
            agg[key] = {"fuerza_agregada": 0.0, "industrias": set()}
        agg[key]["fuerza_agregada"] += fuerza
        agg[key]["industrias"].add(industry)

rows = []
for (exp, imp, inters), vals in agg.items():
    rows.append({
        "exportador": exp,
        "importador": imp,
        "intermediarios": list(inters),  # guardamos como lista para legibilidad
        "fuerza_agregada": vals["fuerza_agregada"],
        "num_industrias": len(vals["industrias"])
    })

caminos_significativos = pd.DataFrame(rows).sort_values(["fuerza_agregada","num_industrias"], ascending=[False,False]).reset_index(drop=True)

out_path = processed_path / "caminos_significativos.parquet"
caminos_significativos.to_parquet(out_path, index=False)
print("✅ caminos_significativos:", caminos_significativos.shape, "→", out_path)
caminos_significativos.head(10)


✅ caminos_significativos: (1066802, 5) → c:\Users\Usuario\Documents\Github\Seguridad Economica\data\processed\dependencias_consolidadas\caminos_significativos.parquet


,exportador,importador,intermediarios,fuerza_agregada,num_industrias
0,ESP,STP,[PRT],32.238254,132
1,ESP,CPV,[PRT],31.297813,139
2,CHN,SWZ,[ZAF],29.827550,113
3,CHN,CXR,[AUS],29.591068,98
4,CHN,LSO,[ZAF],29.209287,115
5,CHN,BTN,[IND],28.533329,110
6,CHN,NIU,[NZL],27.555951,109
7,CHN,BWA,[ZAF],26.782114,115
8,USA,UMI,[CAN],26.547820,83
9,CHN,COK,[NZL],24.175028,121


In [43]:
def peek_parquet(name):
    path = processed_path / name
    dfp = pd.read_parquet(path)
    print(f"📊 {name}: {dfp.shape}")
    return dfp.head(3)

peek_parquet("intermediarios_globales.parquet")
peek_parquet("country_profiles.parquet")
peek_parquet("relaciones_criticas.parquet")
peek_parquet("caminos_significativos.parquet")

print("\n✅ FASE 1 (build foundations) completada.")


📊 intermediarios_globales.parquet: (236, 5)
📊 country_profiles.parquet: (236, 6)
📊 relaciones_criticas.parquet: (7554, 6)
📊 caminos_significativos.parquet: (1066802, 5)

✅ FASE 1 (build foundations) completada.
